In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re

In [ ]:
#i separated the broad and detailed activity data for me. but you can have an excel that has them all but just make sure to reformat the below code
#creating detailed activity and broad sector excel
#change the path to the database file as needed
df = pd.read_excel('broadsector.xlsx', sheet_name='broadsector')
#df = pd.read_excel('detailedactivity.xlsx', sheet_name='broadsector')

#just for broad sector tables
custom_sheet_names = {
    "Economic activity (Broad sector): Agriculture": "Agriculture",
    "Economic activity (Broad sector): Industry": "Industry",
    "Economic activity (Broad sector): Services": "Services",
    "Economic activity (Broad sector): Total": "Total_Sector"
}
# for activity tables
#custom_sheet_names = {
#    "Economic activity (Detailed): Construction ~ISIC rev.4 F": "Construction",
#    "Economic activity (Detailed): Utilities ~ISIC rev.4 D; E": "Utlities",
#    "Economic activity (Detailed): Mining and quarrying ~ISIC rev.4 B": "Mining",
#    "Economic activity (Detailed): Manufacturing ~ISIC rev.4 C": "Mining"

#}

sheets_dict = {}

for classification, short_name in custom_sheet_names.items():
    filtered_df = df[df['classif1.label'] == classification]
    pivot_df = filtered_df.pivot(index='ref_area.label', columns='time', values='obs_value')

    sheets_dict[short_name] = pivot_df

total_df = df.groupby(['ref_area.label', 'time'])['obs_value'].sum().reset_index()
total_pivot = total_df.pivot(index='ref_area.label', columns='time', values='obs_value')

#change broad and detailed activity names as needed
output_path = "broadsectable.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name, data in sheets_dict.items():
        data.to_excel(writer, sheet_name=sheet_name)




In [ ]:
#df = pd.read_excel('broadsectable.xlsx', sheet_name='')
#change sheet name
df = pd.read_excel('detailedacttables.xlsx',sheet_name='Utilities')

#country concordance 
country = pd.read_excel('ILO Country.xlsx')
dt = country.merge(df, on='ILO Country', how='left')
dt = dt.drop(columns='ILO Country')
dt = dt.rename(columns={'IFs Country':'Country'})
dt = dt.rename(columns={'FIPS CODE':'FIPS_CODE'})

year_columns = [col for col in dt.columns if str(col).isdigit()]
dt[year_columns] = dt[year_columns]/1000 
dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

#change decimal place according to the datadict
dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(4)
dt

,Country,FIPS_CODE,1991,1992,1993,1994,1995,1996,1997,1998,...,2016,2017,2018,2019,2020,2021,2022,2023,Earliest,MostRecent
0,Afghanistan,AFG,0.0027,0.0029,0.0032,0.0034,0.0036,0.0037,0.0039,0.0040,...,0.0057,0.0072,0.0106,0.0154,0.0226,0.0197,0.0179,0.0194,0.0027,0.0194
1,Albania,ALB,0.0193,0.0158,0.0172,0.0181,0.0191,0.0193,0.0192,0.0181,...,0.0205,0.0184,0.0172,0.0198,0.0196,0.0197,0.0204,0.0208,0.0193,0.0208
2,Algeria,DZA,0.0993,0.0985,0.1001,0.1018,0.0989,0.1071,0.1147,0.1177,...,0.1085,0.1842,0.1915,0.2040,0.2043,0.2116,0.2257,0.2345,0.0993,0.2345
3,Angola,AGO,0.0133,0.0133,0.0136,0.0132,0.0135,0.0140,0.0152,0.0167,...,0.0304,0.0300,0.0279,0.0274,0.0327,0.0395,0.0408,0.0429,0.0133,0.0429
4,Argentina,ARG,0.1152,0.0961,0.0544,0.0695,0.0718,0.0977,0.0938,0.0845,...,0.1087,0.1512,0.1567,0.1411,0.1482,0.2189,0.1916,0.2082,0.1152,0.2082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,0.0934,0.0980,0.0896,0.0902,0.0880,0.0870,0.0933,0.0819,...,0.0785,0.0540,0.0611,0.0647,0.0645,0.0663,0.0684,0.0729,0.0934,0.0729
184,Viet Nam,VNM,0.0539,0.0551,0.0566,0.0586,0.0595,0.0603,0.0749,0.0871,...,0.2974,0.2929,0.3229,0.3326,0.3408,0.3527,0.3245,0.3387,0.0539,0.3387
185,Yemen,YEM,0.0146,0.0145,0.0138,0.0128,0.0144,0.0143,0.0191,0.0245,...,0.0101,0.0099,0.0105,0.0106,0.0108,0.0110,0.0114,0.0113,0.0146,0.0113
186,Zambia,ZMB,0.0059,0.0060,0.0062,0.0058,0.0062,0.0066,0.0071,0.0078,...,0.0318,0.0317,0.0232,0.0265,0.0208,0.0219,0.0167,0.0176,0.0059,0.0176


In [47]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')
cursor = conn.cursor()
table_name = 'SeriesLaborEmploymentUtilityILO'
columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

dt.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()